# Cuaderno 01: Generación del Dataset Auditivo

**Objetivo:** Crear un dataset de archivos de audio `.wav` a partir de una lista de fonemas/letras clave para español e inglés.

**Flujo de Trabajo:**
1.  **Instalación de Librerías**: Instalar `gTTS` para la síntesis de voz y `pydub` para la conversión de audio.
2.  **Definición de Fonemas**: Crear las listas de sonidos a generar para cada idioma.
3.  **Generación de Audio**: Iterar sobre las listas, generar un archivo `.mp3` para cada fonema usando `gTTS`, y convertirlo al formato `.wav` usando `pydub`.

## Paso 1: Instalación de Librerías

Instalamos las dos librerías principales que necesitamos para este cuaderno.
* `gTTS` (Google Text-to-Speech): Convierte texto en audio.
* `pydub`: Una librería de alto nivel para manipulación de audio, que usaremos para exportar a formato `.wav`.

In [1]:
%pip install gTTS pydub

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.0 MB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (151 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Note: you may need to restart the kernel to use updated packages.


## Paso 2: Configuración y Definición de Listas de Fonemas

Importamos las librerías necesarias, definimos las rutas de salida y creamos las listas de fonemas/letras para cada idioma. Las listas están diseñadas para cubrir los sonidos más representativos.

In [ ]:
import os
from pathlib import Path
from gtts import gTTS
from pydub import AudioSegment

# --- Rutas de Salida ---
project_root = Path.cwd().parent
# Creamos carpetas de salida específicas para cada idioma
output_dir_es = project_root / "data/02_processed/phoneme_audio/es"
output_dir_en = project_root / "data/02_processed/phoneme_audio/en"

output_dir_es.mkdir(parents=True, exist_ok=True)
output_dir_en.mkdir(parents=True, exist_ok=True)


# --- Listas de Fonemas/Letras ---

# Lista para español (incluye vocales, consonantes y dígrafos)
phonemes_es = [
    'a', 'e', 'i', 'o', 'u',
    'b', "c", 'd', 'f', 'g', 'j',"h", 'k', 'l', 'm', 'n', 'ñ', 'p', 'r', 'rr', 's', 't', 'y',"v", 'z',
    'ch', 'll'
]

# Lista para inglés (incluye sonidos de vocales cortas/largas y dígrafos comunes)
phonemes_en = [
    'a', 'e', 'i', 'o', 'u', # Vocales cortas
    'ay', 'ee', 'igh', 'oh', 'oo', # Vocales largas y diptongos
    'b',"c", 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'w', 'y', 'z',
    'ch', 'sh', 'th', 'ng'
]

print(f"Se generarán {len(phonemes_es)} audios para español.")
print(f"Se generarán {len(phonemes_en)} audios para inglés.")

Se generarán 24 audios para español.
Se generarán 32 audios para inglés.


/home/daniel/Proyectos/phonological-awareness/venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
